In [5]:
%load_ext autoreload
%autoreload 2

import dataclasses

import torch
from torch import nn

In [2]:
from src.networks.seq_net import SeqNet
from src.networks.torch_net import TorchNet
net = SeqNet([TorchNet(nn.Sequential(
    nn.ReLU(),
    nn.ReLU(),
))])
print(net.in_features, net.out_features)
net


any same


SeqNet(
  (layers): NetList(
    (0): TorchNet(
      (torch_module): Sequential(
        (0): ReLU()
        (1): ReLU()
      )
    )
  )
)

In [41]:
from src.networks.dense_layer_stack import AdditiveDenseLayerStack, ConcatDenseLayerStack
dense_layer_stack = AdditiveDenseLayerStack(
    lambda _1, _2, _3, _4: nn.Identity(), 
    num_layers=3, 
    num_features=8,
    initial_direct_connection_weight = 3,
    initial_skip_connection_weight = 2,
    connections = 'dense',
    weights_trainable=True,
)
count_parameters(dense_layer_stack)
# dense_layer_stack = ConcatDenseLayerStack(
#     lambda i, is_final_layer, in_size, out_size: nn.Sequential(
#         nn.Linear(in_size, out_size)
#     ),
#     layers_sizes=[5, 3, 7, 2],
#     connections='dense'
# )
# print(dense_layer_stack)
# dense_layer_stack(torch.Tensor([11] * 5), return_dense=True)

128

In [2]:
from src.networks.forward_net import ForwardNet
ForwardNet(
    layer_provider=lambda i, last, in_f, out_f: nn.Sequential(nn.Linear(in_f, out_f), nn.ReLU()),
    layers_in_out_sizes: list[tuple[int, int]] = None,
    layers_sizes: list[int] = None,
    num_layers: int = None,
    num_features: int = None,
    
)

SyntaxError: positional argument follows keyword argument (2220535212.py, line 4)

In [12]:
from src.networks.forward_net import ForwardNet
ForwardNet(layer_provider=lambda _1, _2, _3, _4: nn.Identity(), in_size=1, out_sizes=[2, 3])

ForwardNet(
  (layers): Sequential(
    (0): Identity()
    (1): Identity()
  )
)

In [14]:
from src.networks.fnn import FNN
FNN.self_normalizing(in_size=1, out_sizes=[2, 3])

FNN(
  (layers): Sequential(
    (0): Sequential(
      (0): Linear(in_features=1, out_features=2, bias=True)
      (1): SELU()
    )
    (1): Sequential(
      (0): Linear(in_features=2, out_features=3, bias=True)
    )
  )
)

In [45]:
np.cumsum([1, 2, 3])

array([1, 3, 6])

In [19]:
FNN.self_normalizing(
    1, 
    [2, 5], 
    3, 
    activate_last_layer=True, 
    # dropout_p=0.1, 
    # dropout_last_layer=True
)

FNN(
  (fnn): Sequential(
    (0): Linear(in_features=1, out_features=2, bias=True)
    (1): SELU()
    (2): Linear(in_features=2, out_features=5, bias=True)
    (3): SELU()
    (4): Linear(in_features=5, out_features=3, bias=True)
    (5): SELU()
  )
)

In [35]:
transformer = TransformerEncoder(
    num_layers=8,
    num_features=32,
    attention_num_heads=4,
    feedforward_provider=lambda num_features: FNN(
        input_size=num_features, hidden_sizes=[256], output_size=num_features
    ),
    residual_connection_weight_affine=True,
)
print(count_parameters(transformer))

# nn_transformer = nn.TransformerEncoder(
#     nn.TransformerEncoderLayer(
#         d_model=512,
#         nhead=8,
#     ),
#     num_layers=6
# )
# print(count_parameters(nn_transformer))

# count_parameters(transformer) / count_parameters(nn_transformer)

168704


18914304

1.0

In [5]:
import sympy as sp

In [6]:
a, inv = sp.symbols('a inv')

expr = 1 * a
expr = 2 * expr + 5
expr = 2 ** expr
expr = 3 * expr

inversed = sp.solve(expr - inv, a)[0]
print(inversed)

result = expr.subs(a, 11)

inversed.evalf(subs={inv: result})

log(inv/96)/(2*log(2))


11.0000000000000

In [15]:
%load_ext autoreload
%autoreload 2
import torch
import sympy as sp
from torch import nn
from src.networks.tensor_shape import TensorShape
from src.networks.net import Net

class Tmp(Net):
    
    def __init__(self):
        super().__init__(1, 1)
        self.shape = TensorShape(
            features=sp.sympify(3),
            batch=None,
        )
        features_defined, features = tmp.shape.features_defined
        if features_defined:
        
            self.seq = nn.Sequential(
                *[nn.Linear(3, 3) for _ in range(features)]
            )
        else:
            raise ValueError
    
    def forward(self, x):
        return self.seq(x)
            
        
        
tmp = Tmp()

print(tmp.shape.features_defined)
float(sp.N(tmp.shape.features))

tmp(torch.FloatTensor([1, 2, 3]))

def forward_wrapper(x: torch.Tensor):
    return tmp(x)

torch.jit.script(forward_wrapper)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
(True, 3)


In [62]:
%load_ext autoreload
%autoreload 2
import torch
import sympy as sp
from torch import nn
from src.networks.tensor_shape import TensorShape
from src.networks.net import Net

out_features = sp.symbols('out_features', integer=True)

shape = TensorShape()
shape.features = shape.features * 2 + 1
shape.features = shape.features * 2


in_features = sp.solve(shape.features - out_features, shape.FEATURES_SYMBOL)
for f in in_features:
    print(f.evalf(subs={out_features: int(shape.features.evalf(subs={shape.FEATURES_SYMBOL: sp.Integer(6)}))}))



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
6.00000000000000


In [92]:
shape = TensorShape(3.3)
shape.features *= 2
shape.features % 1 == 0

False

In [12]:
import torch
import torch.nn as nn
import sympy as sp
from typing import Optional

class MyModule(nn.Module):
    
    @torch.jit.ignore
    features: sp.core.Expr
    
    def __init__(
        self,
        features: sp.core.Expr,
        batch: sp.core.Expr,
        # **additional_dimensions: sp.core.Expr,
    ):
        super().__init__()
        self.features = sp.sympify(features)
        self.batch = sp.sympify(batch)
        # self.additional_dimensions = additional_dimensions

    @property
    def features_defined(self) -> tuple[bool, Optional[int]]:
        is_defined = not self.features.free_symbols
        if is_defined:
            features = float(self.features.evalf())
        else:
            features = None
        return is_defined, features
        
    def forward(self, x):
        # Example forward method using the computed initialization value
        return self.coefficient * x

# Example usage
module = MyModule(123, None)
# Since sympy is only used in __init__, this module is compatible with torch.jit.script
scripted_module = torch.jit.script(module)

SyntaxError: invalid syntax (3232634994.py, line 9)

In [36]:
import sympy as sp

f = sp.symbols('f')

expr = f * 3

expr.evalf(subs={sp.symbols('f'): 2})

a, b = sp.symbols('a b')

expr = 2 * (a + 32) ** b
expr.free_symbols

{a, b}

In [2]:
%load_ext autoreload
%autoreload 2
from src.networks.tensor_shape import TensorShape

In [19]:
shape = TensorShape(features=None)#, width=None)
# shape['features'] *= shape['width']
shape['features'] *= 5
# shape = shape.rename_dims({'width': 'length'})

shape2 = TensorShape()
shape2['features'] += 2

print(shape.dimensions)
print(shape2.dimensions)

#shape2.evaluate_forward(shape.evaluate_forward(TensorShape(features=10, length=11))).dimensions
shape.evaluate_forward(shape2).evaluate_forward(TensorShape(features=25)).dimensions

{'features': TensorShape.DimensionInfo(symbol=features, size=5*features), 'batch': TensorShape.DimensionInfo(symbol=batch, size=batch)}
{'features': TensorShape.DimensionInfo(symbol=features, size=features + 2), 'batch': TensorShape.DimensionInfo(symbol=batch, size=batch)}


{'features': TensorShape.DimensionInfo(symbol=features, size=135.000000000000),
 'batch': TensorShape.DimensionInfo(symbol=batch, size=batch)}

In [105]:
a.subs({a: a})

a

In [1]:
%load_ext autoreload
%autoreload 2
from src.networks.core.tensor_shape import TensorShape
from src.networks.core.net import Net
from src.networks.seq_net import SeqNet
from torch import nn
net = SeqNet(nn.Sequential(
    nn.ReLU(),
    nn.ReLU(),
    nn.Dropout(),
    nn.ReLU(),
    nn.Dropout(),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(2, 3),
    nn.ReLU(),
    nn.Dropout(),
    nn.ReLU(),
    nn.Dropout(),
    nn.ReLU(),
    nn.Dropout(),
    nn.ReLU(),
    nn.Linear(3, 5),
    nn.ReLU(),
    nn.Dropout(),
    nn.ReLU(),
    nn.Dropout(),
    nn.ReLU(),
    nn.Dropout(),
    nn.ReLU(),
    nn.Dropout(),
    nn.LayerNorm(5),
    nn.Linear(5, 7),
    nn.ReLU(),
    nn.Conv3d(7, 8, 3),
    nn.Conv3d(8, 8, 3),
    nn.Conv3d(8, 8, 3, dilation=2),
    nn.ReLU(),
    nn.Dropout(),
    nn.ReLU(),
    nn.Dropout(),
    nn.ReLU(),
    nn.Dropout(),
    nn.ReLU(),
    nn.Dropout(),
))
print(net.in_shape)
print(net.out_shape)
print(net.out_shape.evaluate_forward(TensorShape(features=1, s_0=10, s_1=20, s_2=30)))

TensorShape(features = 2.00000000000000, batch = batch)
TensorShape(s_0 = s_0 - 8.0, features = 8.00000000000000, batch = batch, s_2 = s_2 - 8.0, s_1 = s_1 - 8.0)
TensorShape(s_0 = 2.00000000000000, features = 8.00000000000000, batch = batch, s_2 = 22.0000000000000, s_1 = 12.0000000000000)


In [4]:
from torch import nn

nn.Conv2d(8, 2, 3).padding

(0, 0)

In [11]:
from src.networks.parallel_net import ParallelNet
import torch
from torch import nn

ParallelNet.from_layer_provider(
    lambda i, l, in_f, out_f: nn.Linear(in_f, out_f),
    in_size=3,
    out_sizes=[2, 5, 7],
).forward(torch.FloatTensor([1, 2, 3]))

tensor([ 1.0794e+00, -1.9781e+00,  2.1193e-01,  1.3730e+00, -1.6453e-03,
        -2.3417e+00,  6.2371e-01, -3.9108e-01, -1.1929e+00,  7.2638e-01,
        -1.8894e-01,  3.0077e-01,  1.9245e-01,  8.0759e-01],
       grad_fn=<CatBackward0>)